In [9]:
import pandas as pd
import plotly.express as px
import os

min_txs_per_day = 100

In [10]:
df = pd.read_csv('csv_exports/project_deploys_2022_12_09.csv')
# df = pd.read_csv('csv_exports/project_deploys_2022_12_22.csv')

df = df[ (df['txs_per_day'] >= min_txs_per_day)]
# df = df[ (df['type'] == 'Contract Project')]
df['year'] = df['launch_time'].str[:4]
df['date'] = pd.to_datetime( df['launch_time'].str[:10] )

# df.sample(10)

In [11]:
df['type'] = df['type'].str.replace('Contract Project','dApp')
df['type'] = df['type'].str.replace('NFT Only Project','NFT Project')

In [12]:
exclude_list = ['Op','Optimistic Explorer', 'Teleportr/ Warp Speed'] #Exclude OP/ OP Labs Projects

df = df[~df['project'].isin(exclude_list)]

In [13]:
print(df.groupby(by='year').count())
df = df[df['year'] == '2022']
# display(df)

      project  launch_time  num_contracts  type  num_transactions  \
year                                                                
2021       25           25             25    25                25   
2022       54           54             54    54                54   

      txs_per_day  time_elapsed  date  
year                                   
2021           25            25    25  
2022           54            54    54  


In [14]:
fig = px.scatter(df, x="date", y="txs_per_day",
	        #  size="pop", 
                 color="type",
                 hover_name="project",
                #  size_max=60
                labels={"date":"Deployment Date", "txs_per_day":"Transactions per Day", "type":"App Type"}
                ,title = "Projects Deployed on Optimism in 2022 \
                        <br><sup>Project With Greater Than 100 Transactions per Day Since Deployment</sup>"
                , width=800, height=400
                 )
project_annotations = ['Velodrome','Pooltogether','Galxe','Aave','Curve','Beethoven X','Kwenta','Opensea','Atlantis World']
for p in project_annotations:
        fig.add_annotation(x=df[df['project'] == p]['date'].iloc[0], y=df[df['project'] == p]['txs_per_day'].iloc[0],
                text=p,
                showarrow=False,
                yshift=12,
                bgcolor="#E6ECF5",
                opacity=0.9
                )
fig.show()
if not os.path.exists("img_outputs"):
        os.mkdir("img_outputs")
fig.write_image("img_outputs/op_deployments_2022.svg")
fig.write_image("img_outputs/op_deployments_2022.png")

![](https://github.com/MSilb7/op_data/blob/master/img_outputs/op_deployments_2022.svg)

In [15]:
display(df.sort_values(by='txs_per_day', ascending=False))

,project,launch_time,num_contracts,type,num_transactions,txs_per_day,time_elapsed,year,date
144,Velodrome,2022-06-01T20:01:23Z,1812,dApp,2690760.0,14082.399864,191.072546,2022,2022-06-01
285,Galxe,2022-07-18T07:01:41Z,43,dApp,1530600.0,10584.037168,144.614005,2022,2022-07-18
256,Odos,2022-10-06T05:09:31Z,2,dApp,436584.0,6748.665791,64.691898,2022,2022-10-06
215,Granary,2022-05-26T00:38:03Z,64,dApp,493262.0,2492.727721,197.880417,2022,2022-05-26
227,Aave,2022-03-04T21:02:50Z,90,dApp,614757.0,2195.326499,280.029873,2022,2022-03-04
154,Pooltogether,2022-07-01T16:03:46Z,290,dApp,325401.0,2018.146419,161.237558,2022,2022-07-01
117,Beethoven X,2022-04-23T06:02:34Z,311,dApp,413898.0,1794.445801,230.655058,2022,2022-04-23
238,Stargate Finance,2022-03-15T12:57:15Z,42,dApp,459905.0,1707.354122,269.367083,2022,2022-03-15
228,Nex,2022-08-31T06:07:51Z,25,dApp,156190.0,1551.791801,100.651389,2022,2022-08-31
223,Opx Finance,2022-11-02T11:37:57Z,76,dApp,54818.0,1464.854262,37.422153,2022,2022-11-02
